<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Note-for-Chapter-4-Estimating-Point-Correspondence" data-toc-modified-id="Note-for-Chapter-4-Estimating-Point-Correspondence-1">Note for Chapter 4 Estimating Point Correspondence</a></span><ul class="toc-item"><li><span><a href="#From-Photometry-to-Geometry" data-toc-modified-id="From-Photometry-to-Geometry-1.1">From Photometry to Geometry</a></span><ul class="toc-item"><li><span><a href="#Non-rigid-Deformation" data-toc-modified-id="Non-rigid-Deformation-1.1.1">Non-rigid Deformation</a></span></li><li><span><a href="#Small-Deformation-versus-Wide-Baseline" data-toc-modified-id="Small-Deformation-versus-Wide-Baseline-1.1.2">Small Deformation versus Wide Baseline</a></span></li></ul></li><li><span><a href="#Small-Deformation-&amp;-Optical-Flow" data-toc-modified-id="Small-Deformation-&amp;-Optical-Flow-1.2">Small Deformation &amp; Optical Flow</a></span><ul class="toc-item"><li><span><a href="#Small-Deformation" data-toc-modified-id="Small-Deformation-1.2.1">Small Deformation</a></span></li><li><span><a href="#Optic-Flow-Estimation" data-toc-modified-id="Optic-Flow-Estimation-1.2.2">Optic Flow Estimation</a></span></li></ul></li><li><span><a href="#The-Lucas-Kanade-Method" data-toc-modified-id="The-Lucas-Kanade-Method-1.3">The Lucas-Kanade Method</a></span><ul class="toc-item"><li><span><a href="#Estimating-Local-Displacements" data-toc-modified-id="Estimating-Local-Displacements-1.3.1">Estimating Local Displacements</a></span></li><li><span><a href="#When-can-Small-Motion-be-Estimated?" data-toc-modified-id="When-can-Small-Motion-be-Estimated?-1.3.2">When can Small Motion be Estimated?</a></span></li><li><span><a href="#A-Simple-Feature-Tracking-Algorithm" data-toc-modified-id="A-Simple-Feature-Tracking-Algorithm-1.3.3">A Simple Feature Tracking Algorithm</a></span></li></ul></li><li><span><a href="#Feature-Point-Extraction" data-toc-modified-id="Feature-Point-Extraction-1.4">Feature Point Extraction</a></span><ul class="toc-item"><li><span><a href="#Robust-Feature-Point-Extraction" data-toc-modified-id="Robust-Feature-Point-Extraction-1.4.1">Robust Feature Point Extraction</a></span></li></ul></li><li><span><a href="#Wide-Baseline-Matching" data-toc-modified-id="Wide-Baseline-Matching-1.5">Wide Baseline Matching</a></span><ul class="toc-item"><li><span><a href="#Extensions-to-Larger-Baseline" data-toc-modified-id="Extensions-to-Larger-Baseline-1.5.1">Extensions to Larger Baseline</a></span></li><li><span><a href="#Normalized-Cross-Correlation" data-toc-modified-id="Normalized-Cross-Correlation-1.5.2">Normalized Cross Correlation</a></span></li><li><span><a href="#Special-Case:-Optimal-Affine-Transformation" data-toc-modified-id="Special-Case:-Optimal-Affine-Transformation-1.5.3">Special Case: Optimal Affine Transformation</a></span></li></ul></li></ul></li></ul></div>

## Note for Chapter 4 Estimating Point Correspondence

- bilibili视频[lecture6](https://www.bilibili.com/video/av24149942?p=6), [lecture7](https://www.bilibili.com/video/av24149942?p=7)

- 参考书目

```
Ma, Yi, et al. An invitation to 3-d vision: from images to geometric models. Vol. 26. Springer Science & Business Media, 2012.
```

### From Photometry to Geometry

In practice, we do not actually observe points or lines, but rather **brightness or color values** at the individual pixels

主要关注光照和颜色的变化而非直接关注点与线的变化

- photometric representation ==> geometric epresentation
    - 使用特征表达点identify points with characteristic image features
    - 寻找关联性：associate these points with corresponding points

通过关键点的匹配可以帮助探知3D结构，但是，本方法是次优的：选出少量关键点，然后会丢失掉大量可能存在有用信息的特征点（this
approach is suboptimal: By **selecting a small number of feature points** from each image, we **throw away a large amount of potentially useful information** contained in each image）

- Limitations：
    - retaining all image information is computationally challenging
    - tracking of 3D objects from a moving camera in real time - even with limited processing power
    

#### Non-rigid Deformation

在一般算法中，假设物体进行刚体运动：但是现实世界中有可能发生变形（deformation）与遮挡（occlusion）

#### Small Deformation versus Wide Baseline

在点的匹配主要看两种情况——小形变（Small deformation）与（Wide baseline stereo）：

- Small deformation：one frame to the other is assumed to be (infinitesimally) small，使用光流方法解决
    - Lucas/Kanade (sparse)
    - Horn/Schunck (dense)
    one can also track the displacement of a few feature points which is typically faster （跟踪一系列点还可以较快的速度）

- Wide baseline stereo：the displacement is assumed to be large.
    - one typically selects a small number of feature points in each
of the images and develops efficient methods to find an appropriate pairing of points.（需要开发出更为高效的方法去实现实时少数点的匹配）

### Small Deformation & Optical Flow

#### Small Deformation

> 参考书目第70页

one can envision two opposite strategies: 
- **one is to choose a complex transformation that captures the changes undergone by the entire image**, or 
- **one can pick a simple transformation, and then restrict the attention to only those regions in the image whose motion can be captured, within reasonable bounds, by the chosen transformation.**

本部分主要主要关注后面的情况，假设对一静止物体的不同角度进行建模，现有图上对应点$ \mathbf{x_1}, \mathbf{x_2} $；假设两相机间的位移为刚体运动$ (R, T) $（displacement between the two camera viewpoints is a rigid-body motion.），如下所示：
$$
\mathbf{x_2} = h(\mathbf{x_1}) = \frac{1}{\lambda_2(\mathbf{X})}
(R \lambda_1(\mathbf{X}) + T)
$$

在局部则可以对于运动函数$ h $进行建模（The function h describes the transformation of the domain, or "image motion,"）：

- 平移模型Translational model：

$$ h(x) = x + b $$
其中，$ b $为偏移offset

- 仿射模型Affine model：
$$ h(x) = Ax + b $$

![Two basic types of local domain W (x) deformation. Left: translational; right:
affine](./Image/Chap4_affine.jpg)

This model is a good approximation for small
planar patches parallel to the image plane undergoing an arbitrary translation and
rotation about the optical axis, and modest rotation about an axis orthogonal to it.

特别地，2D放射模型可以如下所示：
$$ h(x) = x + u(x)\ , $$
其中
$$ u(x) = S(x)p
= \left( \begin{array}{cccccc}
x & y & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & x & y & 1
\end{array} \right)
\left( \begin{array}{cccccc}
p_1 & p_2 & p_3 & p_4 & p_5 & p_6
\end{array} \right)^{\mathsf{T}}
=
\left( \begin{array}{c}
xp_1 + yp_2 + p_3 \\ xp_4 + yp_5 + p_6
\end{array} \right)
\ . $$

#### Optic Flow Estimation

- **Lucas-Kanade method generates sparse flow vectors under the assumption of constant motion in a local neighborhood**, whereas 
- **the Horn-Schunck method generates a dense flow field under the assumption of spatially smooth motion.**

### The Lucas-Kanade Method

假设：
- Brightness Constancy Assumption

$x(t)$ 表示随时间 $t$ 变化的点，因此视频序列$ l(x,t) $，因此：
$$
l(x(t), t) = \mathrm{const}. \forall t
$$

因而推导出导数应该为0的光流约束方程【(differential) optical flow
constraint】：
$$
\frac{d}{dt}l(x(t), t) = \Delta I^{\mathsf{T}}\left(\frac{dx}{dt} \right) + \frac{\partial I}{\partial t} =0,
\quad
\Delta I^{\mathsf{T}} = \left(\begin{array}{c}
\frac{\partial I}{\partial x} \\
\frac{\partial I}{\partial y}
\end{array}
\right)
$$
其中，$\Delta I^{\mathsf{T}} $为对于图片本身在空间各个位置的导数，$V = \frac{dx}{dt} $可以理解为光流速度速度，此外$ \frac{\partial I}{\partial t} $图片对于时间的限制！**但是公式难以直接解出**

apecture probelm：只能光测出垂直于图片中点位置的运动

- 邻域$ W(x) $内的常速运动Constant motion in a neighborhood
    - 邻域$ W(x) $选择十个难题，一般要出现相关的角度corner
    - 现实中并不适用，传感器的噪音可能有微笑的变化，难以完美实现
    公式如下所示：
    $$
    \Delta I(x',t)^{\mathsf{T}} + \frac{\partial I}{\partial t}(x', t) = 0 \quad \forall x'\in W(x)
    $$

Lucas and Kanade (1981) therefore compute the best velocity vector v for the point x by minimizing the least squares error（最小化二范数），cost (energy) function如下所示：
$$
E(v) = \int_{W(x)} \left| \Delta I(x',t)^{\mathsf{T}} + I_t(x', t) \right|^2 dx'
$$

同样的方法，导数为0：
$$
\frac{dE}{dc} = 2Mv + 2q =0,
$$
其中，
$$
M = \int_{W(x)} \Delta I \Delta I^{\mathsf{T}} dx', 
\quad and \quad 
q = \int_{W(x)} I_t \Delta I dx'.
$$

若$ \mathrm{det}(M) \not= 0 $，那么问题的解可以得出：$ v = -M^{-1} q $。

#### Estimating Local Displacements

同样分为两种模型：

- Translational motion: Lucas & Kanade ’81:

最小化平方和，如下所示：
$$
E(b) = \int_{W(x)} \left| \Delta I^{\mathsf{T}}b + I_t\right|^2 dx'
\rightarrow \mathrm{min}\ .
$$

然后对于$b$求导得到结果！

- Affine motion:

使用**Small Deformation**假设下的模型最小化平方和，如下所示：
$$
E(b) = \int_{W(x)} \left| \Delta I(x')^{\mathsf{T}} S(x')p + I_t(x') \right|^2 dx'
\rightarrow \mathrm{min}\ .
$$

然后对于$p$求导得到结果！

以上$b$与$p$均为向量表示！

#### When can Small Motion be Estimated?

> [孔径问题(aperture problem.)](https://blog.csdn.net/hankai1024/article/details/23433157)

Lucas-Kanade会遇到孔径问题，如遇到白墙（if the region
in the window W(x) around the point x has entirely constant
intensity (for example a white wall)），在这种情况下：所有在窗内的点对应$ \Delta I(x) = 0 $ 与 $ I_t(x) = 0 $或接近于。

因此，structure tensor/matrix即：
$$
M(x) = \int_{W(x)}
\left(
\begin{array}{cc}
I_x^2  & I_xI_y \\
I_xI_y & I_y^2
\end{array}
\right)
$$
必须是可逆的。

若$ \mathrm{det}(M) $尽管不可逆但不等于，还是可以对法向运动进行估计。(estimate the normal motion, i.e. the motion in direction of the image gradient)

进而到处以下的特征跟踪器（This leads to the following simple feature tracker）

#### A Simple Feature Tracking Algorithm

- 对于每个时间片段，计算每个点的Structure tensor (For a given time instance t, compute at each point $ x \in \Omega $ the structure tensor)：
$$
M(x) = \int_{W(x)}
\left(
\begin{array}{cc}
I_x^2  & I_xI_y \\
I_xI_y & I_y^2
\end{array}
\right)
$$

- 选取对应矩阵行列式$ \mathrm{det}(M) $超过阈值的点$ \theta > 0$

- 计算对应速度：
$$
b(x, t) = -M(x)^{-1} \left(
\begin{array}{c}
\int I_x I_t dx' \\
\int I_y I_t dx'
\end{array}
\right)\ .
$$

- 然后重复上述步骤。

**局限**：如果邻域间的光度变化不是非常的大，那么可能造成$I_x$与$I_y$非常小，可能无法超过

### Feature Point Extraction

#### Robust Feature Point Extraction

**局限**：矩阵逆可能十分不稳定（the inverse of M(x) may not be very stable），从而导致在一些情况下，M的行列式非常小，因此有些情况$\mathrm{det} (M) \not= 0 $不一定适合跟踪。

**解决方案**：开发出相关经典特征子如Moravec ’80, Förstner ’84, ’87 and Harris & Stephens ’88。

其中一种方法：就是在$ M(x) $上加上与特征点中心相关的高斯分布（Gaussian distribution）实现基于高斯的权重分配：
$$
M(x) 
=G_{\sigma} \star \Delta I \Delta I^{\mathsf{T}}
=\int_{W(x)}
G_{\sigma}(x-x')
\left(
\begin{array}{cc}
I_x^2  & I_xI_y \\
I_xI_y & I_y^2
\end{array}
\right)
(x')dx'
$$

Harris提出了如下方法：
$$
C(x) = \mathrm{det}(M) - \kappa mathrm{trace}^{2}(M)\ ,
$$
然后选取其中$ C(x) > \theta $进行二次筛选，找到更为健壮robust的点。

### Wide Baseline Matching

唯一非常大，难以直接匹配（displacement is very much
many points cannot match
substantial rotation）

large parts of the image plane will not match at all because they are not visible in the other image. In other words, while a given point may have many potential matches, quite possibly it does not have a corresponding point in the other image.

#### Extensions to Larger Baseline

跟踪一大局限：隔帧之间的差别在大部分情况下非常小，因此可能不断积累小错误One of the limitations of tracking features frame by frame is that small errors in the motion accumulate over time。

其中一个方法就是与第一帧进行比对，找到相对更大的区别，或者选取关键帧。A remedy is to **match a given point back to the first frame**. This generally implies **larger displacements between frames.**

并且有两个方面的想法去解决大偏移：

- motion of the window between frames is **(in general) no longer translational**, for example by using an **affine motion model**.

- replace the sum-of-squared-differences by the **normalized cross correlation** which is more **robust to illumination changes**

#### Normalized Cross Correlation

主要定义如下：
$$
\mathrm{NCC}(h)
= \frac
{
\int_{W(x)} 
\left( I_1(x') - \overline{I_1}\right)
\left( I_2(h(x')) - \overline{I_2}\right) dx'
}
{\sqrt{\int_{W(x)} 
\left( I_1(x') - \overline{I_1}\right)^2 dx'
\int_{W(x)}\left( I_2(h(x')) - \overline{I_2}\right)^2 dx'}}
$$

- 通过减去平均值，可以对于额外的增量不变（By subtracting this average intensity, the measure becomes invariant to additive intensity changes） $ I \rightarrow I + \gamma $

- 通过除以方差，可以对于成倍的光照变化不变（Dividing by the intensity variances of each window makes the
measure invariant to multiplicative changes） $ I \rightarrow \gamma I  $

❗ 对于本页后续部分，还没理解待补充

#### Special Case: Optimal Affine Transformation

已知有仿射模型：
$$
h(x) = Ax + d,
$$
然后最大化正则互相关，对一直矩阵，如下所示：
$$
\hat{A}, \hat{d} = \arg \max_{A,d} NCC(A,d}
$$

然后将上述模型中的$ h(x') $替换成$Ax' + d$仿射模型，但是求解有一定难度（Efficiently finding appropriate optima, however, is a challenge.）